In [272]:
from wield.control import SISO
from wield.control.AAA import tfAAA
from wield.iirrational.v2 import data2filter

import wieldMoku

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
from moku.instruments import MultiInstrument, LaserLockBox, DigitalFilterBox, FrequencyResponseAnalyzer, SpectrumAnalyzer

# Set up multinstrument mode skeleton

In [273]:
MIM, llb, fra, dfb, spa = wieldMoku.standard_MultiInstrument('192.168.50.97') #192.168.50.97 moku pro 1

In [274]:
propGain = 0 #in dB
intSaturation = 40.0 #in dB
intCrossover = 18e3 #in Hz

K_model = wieldMoku.set_controller(llb,propGain,intCrossover,intSaturation)

# Run cell below until lock is aquired

In [302]:
wieldMoku.aquire_lock(llb)

In [276]:
raise ValueError('breakpoint')

ValueError: breakpoint

## When locked take transfer function

In [ ]:
#configure the frequency sweep
fra.set_sweep(start_frequency=200, stop_frequency=10e6, num_points=256,
                averaging_time=1e-3, averaging_cycles=100, settling_cycles=10,
                settling_time=1e-3,dynamic_amplitude = True)
fra.measurement_mode('InIn1')
fra.set_output(1, 0.005)

delay = fra.start_sweep() 
print(delay)
data = pd.DataFrame(fra.get_data(wait_complete = True)['ch2'])

fra.stop_sweep()

{'estimated_sweep_time': 43.043360218801766}


In [ ]:
fra.stop_sweep()

In [ ]:
data['complex'] = data.apply(lambda x: 10**(x.magnitude/20)*np.exp(1.0j*x.phase*np.pi/180), axis=1)

OpenLoopFResponse = SISO.SISOFResponse(f = data['frequency'],tf = data['complex'])

In [ ]:
PlantFResponse = OpenLoopFResponse/K_model.fresponse(f = OpenLoopFResponse.f)

In [ ]:
idx = np.round(np.linspace(0, len(PlantFResponse.f) - 1, 50)).astype(int)

downsampledPlant = SISO.SISOFResponse(f = PlantFResponse.f[idx],tf = PlantFResponse.tf[idx])
barycentric = tfAAA(F_Hz=downsampledPlant.f, xfer=downsampledPlant.tf)
AAAfit = SISO.zpk(barycentric.zpk)

/opt/python-dev/wield-project-lab/wield-control/src/wield/control/SISO/zpk.py:134: NumericalWarning: StateSpace is large (>50 states), using reduced response fiducial auditing heuristics. TODO to make this smarter
  warnings.warn(f"StateSpace is large (>{self.N_MAX_FID} states), using reduced response fiducial auditing heuristics. TODO to make this smarter", util.NumericalWarning)


In [ ]:
iir_results = data2filter(
    F_Hz=downsampledPlant.f,#from a few cells ago
    xfer=downsampledPlant.tf,
    mode='reduce',
    zeros=AAAfit.z, 
    poles=AAAfit.p,
    gain=AAAfit.k,
    SNR_phase_rel=0,
    SNR=np.ones_like(downsampledPlant.f),
    relative_degree=-4,
    # resavg_RthreshOrdDn=1.01,
    baseline_only=True,
    # coding_map=fitters_ZPK.codings_s.coding_maps.RI
    # trust_SNR = True,
    )

/opt/python-dev/wield-project-lab/wield-iirrational/src/wield/iirrational/fitters_ZPK/MRF.py:1361: RuntimeWarning: divide by zero encountered in scalar divide
  ratio = y_min / y_max


TEE_LOGFILE None
------------:Q-ranked order reduction:
4P   1.01    order reduced annealing


3W   0.99    Fitter_checkpoint improvement succeed, None


5P   1.39  zero flipping, maxzp 19, residuals=1.19e-01, 1.69e-01, reldeg=0
------------:selective order reduction:
5P   6.71    order reduced to 18, residuals=5.91e-02, reldeg=-1
5P  10.19    order reduced to 18, residuals=1.28e-02, reldeg=-2
5P  14.18    order reduced to 16, residuals=9.80e-03, reldeg=-2
2A  15.52  Baseline fit residuals: 9.80e-03, at order 16
BASELINE:  16
------------:investigations:
2I  15.53    max(z, p)       ChiSq.
                   order    avg. res.    med. res.    max. res.
             -----------  -----------  -----------  -----------
                      16   0.00979857  0.000408133    0.0276528
                      31  -0.0135318   0.000503136    0.043196


3W  15.51  Fitter_checkpoint improvement succeed, None


In [ ]:
z = iir_results.fitter.zeros.fullplane
p = iir_results.fitter.poles.fullplane
k = iir_results.fitter.gain
Plant = SISO.zpk(z,p,k,convention = 'IIRrational')

In [ ]:
P = Plant.fresponse(f = OpenLoopFResponse.f)

In [313]:
def dataframe(fresponse,label):
    return pd.DataFrame(dict(frequency = fresponse.f,magnitude = fresponse.mag, phase = fresponse.rad*180/np.pi,label = label))

In [ ]:
#turn into dataframes for plotting
alldata = pd.concat(
    [dataframe(OpenLoopFResponse,"measured open loop"),
     dataframe(K_model.fresponse(f = OpenLoopFResponse.f),"controller"),
     dataframe(PlantFResponse,"plant"),
     dataframe(Plant.fresponse(f = OpenLoopFResponse.f),"plant model")])

In [ ]:
magnitudechart = alt.Chart(alldata).mark_line().encode(
    x=alt.X('frequency:Q').scale(type="log"),
    y=alt.Y('magnitude:Q').scale(type="log"),
    color='label:N',
).properties(
    width=400,
    height=200
)

phasechart = alt.Chart(alldata).mark_line().encode(
    x=alt.X('frequency:Q').scale(type="log"),
    y=alt.Y('phase:Q'),
    color='label:N',
).properties(
    width=400,
    height=100
)

chart = alt.vconcat(magnitudechart,phasechart)
chart

alt.VConcatChart(...)

# Now we take a spectrum

In [303]:
spa.sa_measurement(channel = 1, frequency1 = 10, frequency2 = 100e3)
data = pd.DataFrame(spa.get_data(units = 'Vrms',psdUnits = False,wait_complete=True))

In [309]:
data= data.drop(0,axis=0)

In [311]:
controlASD = SISO.SISOFResponse(f = data['frequency'],tf = data['ch1'])

In [312]:
errorASD = controlASD/K_model.fresponse(f = controlASD.f)

In [315]:
allspectra = pd.concat(
    [dataframe(controlASD,"control ASD"),
     dataframe(errorASD,"error ASD")])

In [316]:
alt.Chart(allspectra).mark_line().encode(
    x=alt.X('frequency:Q').scale(type="log"),
    y=alt.Y('magnitude:Q').scale(type="log"),
    color='label:N'
).properties(
    width=400,
    height=200
)

alt.Chart(...)

In [ ]:
MIM.relinquish_ownership()